# 1. Doc2Vec als Speziellkategorie von Skills2Vec

In [12]:
pip install --upgrade --force-reinstall gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 66.1 MB/s eta 0:00:00
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.3.0
    Uninstalling smart-open-6.3.0:
      Successfully uninstalled smart-open-6.3.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstallin

In [9]:
pip install --upgrade --force-reinstall numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import files
 
 
uploaded = files.upload()

Saving naukri_data_science_jobs_india_cleaned.csv to naukri_data_science_jobs_india_cleaned.csv


In [4]:
import pandas as pd

In [18]:
def helper(column):
    return str(column).split(",")

def open_csv_default():
  df_data = pd.read_csv("naukri_data_science_jobs_india.csv")
  df_data = df_data.drop(columns=['Company', 'Location', 'Job Experience'])
  df_data["Skills/Description"] =df_data["Skills/Description"].apply(lambda x : helper(x))
  
  return df_data

def open_csv_clean():
  df_data = pd.read_csv("naukri_data_science_jobs_india_cleaned.csv")
  #df_data = df_data.drop(columns=['Company', 'Location', 'Job Experience'])
  #df_data["Skills/Description"] =df_data["Skills/Description"].apply(lambda x : helper(x))
  return df_data

In [106]:
### Option 1: Use Doc2Vec which includes the "document title" in learning.
## Works out of the box

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import ast

#data = open_csv_default()
data = open_csv_clean()

# Preprocess the data and create TaggedDocuments
#tagged_data = [TaggedDocument(words=skills, tags = [job]) for skills in data]
#tagged_data = [TaggedDocument(words=skills, tags=[job]) for job, skills in data]
tagged_data = [TaggedDocument(words=ast.literal_eval(skills), tags=[job]) for job, skills in data[:]]

# Train the Doc2Vec model
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(tagged_data)

model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)



In [101]:
import numpy

# Test the model
#print(model.wv["python"])
print(numpy.linalg.norm(model.wv["advanced_analytics"] - model.wv["analytics"]))
print(numpy.linalg.norm(model.wv["machine_learning"] - model.wv["management"]))

# Enter skills and get most suitable jobs
skills = ['machine_learning', 'advanced_analytics']
infer_vector = model.infer_vector(skills)
similar_jobs = model.dv.most_similar([infer_vector], topn=5)

print(f"Given skills: {skills}")
print("Most suitable jobs:")
for job, similarity in similar_jobs:
    print(f"- {job} (Similarity: {similarity:.2f})")


5.1436515
7.944127
Given skills: ['machine_learning', 'advanced_analytics']
Most suitable jobs:
- Data Science – Delivery Lead (Similarity: 0.96)
- Product Manager - Advanced Analytics & AI (Similarity: 0.96)
- Business Process Analyst (Similarity: 0.96)
- Manager - Data Scientist Job @ Applied Materials - Bangalore (Similarity: 0.95)
- Senior Data Scientist - Regression Modeling (Similarity: 0.95)


In [41]:
### Training uses the Doc2Vec or Word2Vec model which simply embeds words according to how often they occur together (ignoring order afaik)
## Adapt vector size and epochs in training once data set is complete

from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import ast
import numpy

def trainWord2Vec(data):
# Preprocess the data and create list of skills (this training does NOT include jobs!)
  tagged_data = [ast.literal_eval(skills) for job, skills in data[:]]

# Train the Word2Vec model, try different vector sizes for interesting effects in similarities
  model = Word2Vec(vector_size=50, min_count=1, workers=4, epochs=20)
  model.build_vocab(tagged_data)

  model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

  return model

def trainDoc2Vec(data):
  tagged_data = [TaggedDocument(words=ast.literal_eval(skills), tags=[job]) for job, skills in data[:]]

# Train the Doc2Vec model
  model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
  model.build_vocab(tagged_data)

  model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

  return model

# helper method to calculate average embedding vector of a list of string skills
def averageOfSkills(model, list, axis = 0):
  return numpy.average([model.wv[i] for i in list], axis = axis)

In [7]:
import numpy

def calcAvgEmbeddings(model):

  # Tests for the embedding go here
  print(model.wv.most_similar("neural_networks", topn = 5))
#cosine distance
  print(model.wv.distance("finance", "machine_learning"))
  print(model.wv.distance("ai", "machine_learning"))
  print(model.wv.distance("business_intelligence", "management"))
  print(model.wv.distance('corporate_governance', 'ai'))
  #euclidean distance
  #print(numpy.linalg.norm(model.wv["statistics"] - model.wv["machine_learning"]))

  # split the tuples
  job_titles, skills = list(zip(*data))
  # calculate the average skill vector of every job offering / person and zip back together
  skillAverages = [averageOfSkills(model, ast.literal_eval(skill)) for skill in skills]
  data_averaged = list(zip(job_titles, skillAverages))

  return (skillAverages, data_averaged)

# This gives us a list of job offerings and average embeddings. 
# Could be used as input to a graph neural network or knowledge graph?
# Is used below to classify immediately

In [43]:
# simply suggest smallest distance to user's average ("Option 2") using word2Vec
import itertools
import operator
from scipy.spatial.distance import cdist

def prepareOption2(data):

  model = trainWord2Vec(data)
  skillAverages, data_averaged = calcAvgEmbeddings(model)

  # sort before grouping
  data_averaged.sort(key=operator.itemgetter(0))
  # Group by job title, take the average embedding of everyone with that title and make a dictionary (maps job title to average embedding)
  job_averages = {key : numpy.average(list(zip(*list(value)))[1], axis = 0)
    for key, value in itertools.groupby(data_averaged, lambda x: x[0])}

  #keep keys and values
  keys = list(job_averages.keys())
  values = list(job_averages.values())
  # easy access to the avg vector
  #print(job_averages['Advisor, Data Science'])

  return job_averages, keys, values, model

# usage in next cells

In [22]:
# "Option 3" could be using the embedding learned by Doc2Vec and doing the same manual averaging as Option 2 for "learning" the correlation of job to skills
import itertools
import operator
from scipy.spatial.distance import cdist

def prepareOption3(data):
#if !data:
#  data = open_csv_clean()

  model = trainDoc2Vec(data)
  skillAverages, data_averaged = calcAvgEmbeddings(model)

  # sort before grouping
  data_averaged.sort(key=operator.itemgetter(0))
  # Group by job title, take the average embedding of everyone with that title and make a dictionary (maps job title to average embedding)
  job_averages = {key : numpy.average(list(zip(*list(value)))[1], axis = 0)
    for key, value in itertools.groupby(data_averaged, lambda x: x[0])}

  #keep keys and values
  keys = list(job_averages.keys())
  values = list(job_averages.values())

  return job_averages, keys, values, model

In [16]:
# USAGE of Option 2 and 3:
# (not a proper test yet)

data = open_csv_clean()
formatted = data.values.tolist()
job_averages, keys, values, model = prepareOption3(data)
#job_averages, keys, values, model = prepareOption2(data)

#Given some skill list:
#my_skills = ["business_intelligence", "management"]
my_skills = ['python', 'tableau', 'machine_learning', 'sql', 'statistical_modeling', 'adv_excel', 'microstrategy', 'r_shiny']
# we can calculate the average embedding
avg = averageOfSkills(model, my_skills)

#and match it to the closest job title
# calculate distance to all jobs for lack of better solution, optimize this if there's time
distVec = cdist([avg], values)
# zip the distances with the keys and sort jobs by distance
sortedList = sorted(list(zip(distVec.transpose(), keys)))
# print top 5 scoring jobs (i.e. smallest distance)
result = [("" + str(i) +": Score " + str(1 / max(scor[0], 0.1))) for scor,i in sortedList[0:5]]
#result = [("" + str(i) +": Score " + str(scor)) for scor,i in sortedList[0:5]]
print("Top Jobs: ")
for i in range(5):
  print(result[i])

# Our model option 2 should immediately support another application: Find top 10 skills of some job

job = "Data Manager"

embed = job_averages[job]

embed = embed / numpy.linalg.norm(embed)

distVec = cdist([embed], model.wv.get_normed_vectors())

print(distVec.shape)
print(len(distVec))

sortIndicesByDist = sorted(list(zip(distVec.transpose(), range(len(distVec.transpose())))))
result = [(model.wv.index_to_key[index], score) for score,index in sortIndicesByDist[0:10]]

print("Main skills for "+ job)
for i in range(10):
  print(result[i])


Top Jobs: 
Sr. Analyst - Applied Data Scientist: Score 10.0
Senior Business Analyst - Analytics Consulting | Tiger Analytics |: Score 1.069277429627183
Digital Reporting Factory Analyst - Power BI & Power Apps: Score 0.9821993462743488
Data Engineer/data Science Consultant: Score 0.9775174614089209
Senior Associate-Business Analyst: Score 0.9343557347633233
(1, 6958)
1
Main skills for Data Manager
('instrumentation', array([0.44565565]))
('strategic_sourcing', array([0.44704109]))
('schema', array([0.45471818]))
('silicon', array([0.45573203]))
('healthcare_analytics', array([0.45765593]))
('resource_management', array([0.45960371]))
('team_building', array([0.46047742]))
('engineering_manager', array([0.46481943]))
('script_writing', array([0.46781492]))
('principal', array([0.47362737]))


In [52]:
# Proper testing
import heapq

# divide data into 75 train, 25 test
def train_test_split(df, frac=0.25):
    
    # get random sample 
    test = df.sample(frac=frac, axis=0)

    # get everything but the test sample
    train = df.drop(index=test.index)

    return train, test

df = open_csv_clean()

df_data, df_test = train_test_split(df)

formatted_data = df_data.values.tolist()
formatted_test = df_test.values.tolist()
jobs, skills = list(zip(*formatted_test))
skills = [ast.literal_eval(skill) for skill in skills]

print(len(skills))

option = 2
if option == 1:
  model = trainDoc2Vec(formatted_data)
  correct = 0
  top5 = 0
  for i in range(len(jobs)):
    infer_vector = model.infer_vector(skills[i])
    similar_jobs = model.dv.most_similar([infer_vector], topn=5)
    if(similar_jobs[0][0] == jobs[i]):
      correct = correct + 1
      top5 = top5 + 1
    elif(any(item[0] == jobs[i] for item in similar_jobs)):
      top5 = top5 + 1
  resultCorrect = correct / len(jobs)
  resultTop5 = top5 / len(jobs)
  print("Option 1: Percent of correct guesses: " + str(resultCorrect) + ", percent of top 5 guesses: " + str(resultTop5))
elif option == 2:
  job_averages, keys, values, model = prepareOption2(data)
  correct = 0
  top5 = 0
  for i in range(len(jobs)):
    avg = averageOfSkills(model, skills[i])
    distVec = cdist([avg], values)
    #sortedList = sorted(list(zip(distVec.transpose(), keys)))
    #topJobs = [i for scor,i in sortedList[0:5]]
    topJobs = [k for dist, k in heapq.nsmallest(5, zip(distVec.transpose(), keys))]
    if(topJobs[0] == jobs[i]):
      correct = correct + 1
      top5 = top5 + 1
    elif(jobs[i] in topJobs):
      top5 = top5 + 1
  resultCorrect = correct / len(jobs)
  resultTop5 = top5 / len(jobs)
  print("Option 2: Percent of correct guesses: " + str(resultCorrect) + ", percent of top 5 guesses: " + str(resultTop5))

3000
[('text_mining', 0.9697443842887878), ('computer_vision', 0.9461206197738647), ('deep_learning', 0.9408956170082092), ('image_processing', 0.9356536269187927), ('time_series_analysis', 0.9239935278892517)]
0.8608904480934143
0.48566001653671265
0.4894871115684509
0.5936182737350464
Option 2: Percent of correct guesses: 0.5373333333333333, percent of top 5 guesses: 0.5866666666666667


In [36]:
print(skills[0])

['customer_service', 'troubleshooting', 'written_communication', 'verbal', 'business_process', 'it_services', 'analyst', 'automation']


In [6]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# Assuming you have a list of job titles and corresponding skills
data = [
    {
      "job_title": "Production Supervisor",
      "skills": [
        "Production planning",
        "Inventory management",
        "Quality control",
        "Team leadership",
        "Lean manufacturing",
        "Process improvement",
        "Safety regulations",
        "Problem-solving",
        "Communication",
        "Time management",
        "Six Sigma",
        "Supply chain management"
      ]
    },
    {
      "job_title": "Quality Assurance Technician",
      "skills": [
        "Quality inspections",
        "Testing procedures",
        "Statistical analysis",
        "Root cause analysis",
        "Documentation",
        "ISO standards",
        "Data analysis",
        "Attention to detail",
        "Technical knowledge",
        "Report writing",
        "Audit procedures",
        "Corrective actions",
        "Problem-solving"
      ]
    },
    {
      "job_title": "Machine Operator",
      "skills": [
        "Machine setup",
        "Equipment maintenance",
        "Troubleshooting",
        "Safety protocols",
        "Quality checks",
        "Blueprint reading",
        "Precision measurement",
        "Material handling",
        "Workplace organization",
        "Adherence to schedules",
        "Problem-solving",
        "Attention to detail"
      ]
    },
    {
      "job_title": "Supply Chain Coordinator",
      "skills": [
        "Inventory management",
        "Order processing",
        "Logistics coordination",
        "Supplier management",
        "Data analysis",
        "Forecasting",
        "Vendor communication",
        "SAP/ERP systems",
        "Problem-solving",
        "Negotiation skills",
        "Organizational skills",
        "Attention to detail",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Maintenance Technician",
      "skills": [
        "Equipment repair",
        "Preventive maintenance",
        "Troubleshooting",
        "Electrical systems",
        "Mechanical systems",
        "PLC programming",
        "Hydraulics",
        "Pneumatics",
        "Blueprint reading",
        "Safety regulations",
        "Problem-solving",
        "Communication",
        "Teamwork"
      ]
    },
    {
      "job_title": "Assembly Line Worker",
      "skills": [
        "Product assembly",
        "Hand tool operation",
        "Quality checks",
        "Workplace safety",
        "Standard operating procedures",
        "Teamwork",
        "Attention to detail",
        "Physical stamina",
        "Time management",
        "Adaptability",
        "Problem-solving"
      ]
    },
    {
      "job_title": "Industrial Engineer",
      "skills": [
        "Process optimization",
        "Time study",
        "Layout design",
        "Workstation ergonomics",
        "Material flow analysis",
        "Simulation software",
        "Project management",
        "Data analysis",
        "Lean manufacturing",
        "Six Sigma",
        "Problem-solving",
        "Communication",
        "Leadership"
      ]
    },
    {
      "job_title": "Packaging Operator",
      "skills": [
        "Packaging machinery",
        "Quality checks",
        "Labeling",
        "Material handling",
        "Safety protocols",
        "Attention to detail",
        "Workplace organization",
        "Adherence to procedures",
        "Time management",
        "Teamwork",
        "Problem-solving"
      ]
    },
    {
      "job_title": "Forklift Operator",
      "skills": [
        "Forklift operation",
        "Material handling",
        "Inventory management",
        "Safety regulations",
        "Order picking",
        "Loading and unloading",
        "Attention to detail",
        "Time management",
        "Communication",
        "Problem-solving"
      ]
    },
    {
      "job_title": "Process Technician",
      "skills": [
        "Process monitoring",
        "Equipment calibration",
        "Data collection",
        "Quality checks",
        "Troubleshooting",
        "Documentation",
        "Safety protocols",
        "Attention to detail",
        "Analytical skills",
        "Communication",
        "Problem-solving",
        "Teamwork"
      ]
    },
    {
      "job_title": "CNC Machinist",
      "skills": [
        "CNC programming",
        "Machine setup",
        "Tooling selection",
        "Precision machining",
        "Blueprint reading",
        "Quality control",
        "Troubleshooting",
        "Safety protocols",
        "Material handling",
        "Attention to detail",
        "Problem-solving",
        "Communication",
        "Time management"
      ]
    },
    {
      "job_title": "Process Engineer",
      "skills": [
        "Process optimization",
        "Equipment design",
        "Data analysis",
        "Statistical process control",
        "Quality assurance",
        "Process documentation",
        "Project management",
        "Troubleshooting",
        "Root cause analysis",
        "Problem-solving",
        "Communication",
        "Teamwork",
        "Technical knowledge"
      ]
    },
    {
      "job_title": "Warehouse Supervisor",
      "skills": [
        "Inventory management",
        "Order processing",
        "Shipping and receiving",
        "Warehouse organization",
        "Forklift operation",
        "Safety regulations",
        "Team leadership",
        "Communication",
        "Problem-solving",
        "Time management",
        "Attention to detail",
        "Supervisory skills",
        "ERP systems"
      ]
    },
    {
      "job_title": "Electrical Engineer",
      "skills": [
        "Electrical system design",
        "Circuit analysis",
        "PLC programming",
        "Control systems",
        "Troubleshooting",
        "Safety regulations",
        "Project management",
        "AutoCAD",
        "Technical documentation",
        "Problem-solving",
        "Communication",
        "Teamwork",
        "Analytical skills"
      ]
    },
    {
      "job_title": "Logistics Coordinator",
      "skills": [
        "Supply chain management",
        "Inventory control",
        "Logistics coordination",
        "Transportation management",
        "International shipping",
        "Customs regulations",
        "Data analysis",
        "ERP systems",
        "Problem-solving",
        "Communication",
        "Organizational skills",
        "Time management",
        "Attention to detail"
      ]
    },
    {
      "job_title": "Safety Specialist",
      "skills": [
        "Safety regulations",
        "Risk assessment",
        "Safety training",
        "Accident investigation",
        "OSHA compliance",
        "Emergency preparedness",
        "Safety audits",
        "Hazard identification",
        "Communication",
        "Problem-solving",
        "Attention to detail",
        "Teamwork",
        "Analytical skills"
      ]
    },
    {
      "job_title": "Quality Control Inspector",
      "skills": [
        "Quality inspections",
        "Testing procedures",
        "Dimensional measurement",
        "Blueprint reading",
        "ISO standards",
        "Documentation",
        "Root cause analysis",
        "Problem-solving",
        "Attention to detail",
        "Communication",
        "Data analysis",
        "Report writing"
      ]
    },
    {
      "job_title": "Research and Development Engineer",
      "skills": [
        "Product design",
        "Prototype development",
        "Testing procedures",
        "Data analysis",
        "CAD software",
        "Project management",
        "Problem-solving",
        "Technical documentation",
        "Innovation",
        "Communication",
        "Teamwork",
        "Analytical skills",
        "Time management"
      ]
    },
    {
      "job_title": "Inventory Analyst",
      "skills": [
        "Inventory management",
        "Demand forecasting",
        "Data analysis",
        "Supplier coordination",
        "Order processing",
        "ERP systems",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Analytical skills",
        "Teamwork",
        "Time management"
      ]
    },
    {
      "job_title": "Automation Engineer",
      "skills": [
        "Automation systems",
        "PLC programming",
        "Robotics",
        "Machine vision",
        "Control systems",
        "Troubleshooting",
        "Project management",
        "Process optimization",
        "Safety protocols",
        "Problem-solving",
        "Communication",
        "Teamwork",
        "Technical knowledge"
      ]
    },
    {
      "job_title": "Materials Planner",
      "skills": [
        "Material requirement planning",
        "Inventory management",
        "Supply chain coordination",
        "Demand forecasting",
        "SAP/ERP systems",
        "Data analysis",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Organizational skills",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Process Operator",
      "skills": [
        "Process monitoring",
        "Equipment operation",
        "Safety protocols",
        "Quality control",
        "Troubleshooting",
        "Standard operating procedures",
        "Attention to detail",
        "Communication",
        "Problem-solving",
        "Teamwork",
        "Time management"
      ]
    },
    {
      "job_title": "Technical Writer",
      "skills": [
        "Technical documentation",
        "User manuals",
        "Standard operating procedures",
        "Editing and proofreading",
        "Content management systems",
        "Attention to detail",
        "Communication",
        "Problem-solving",
        "Time management",
        "Organizational skills",
        "Teamwork",
        "Subject matter expertise"
      ]
    },
    {
      "job_title": "Quality Engineer",
      "skills": [
        "Quality management systems",
        "Statistical process control",
        "Root cause analysis",
        "Process improvement",
        "Problem-solving",
        "Auditing",
        "Quality assurance",
        "ISO standards",
        "Documentation",
        "Data analysis",
        "Communication",
        "Project management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Shipping and Receiving Clerk",
      "skills": [
        "Shipping and receiving",
        "Inventory management",
        "Order processing",
        "Forklift operation",
        "Data entry",
        "Attention to detail",
        "Communication",
        "Problem-solving",
        "Time management",
        "Organizational skills",
        "Teamwork"
      ]
    },
    {
      "job_title": "Lean Manufacturing Specialist",
      "skills": [
        "Lean manufacturing principles",
        "Process optimization",
        "Value stream mapping",
        "Kaizen",
        "5S methodology",
        "Kanban",
        "Problem-solving",
        "Data analysis",
        "Project management",
        "Communication",
        "Teamwork",
        "Change management",
        "Continuous improvement"
      ]
    },
    {
      "job_title": "Mechanical Engineer",
      "skills": [
        "Mechanical design",
        "CAD software",
        "Engineering drawings",
        "Material selection",
        "Prototype development",
        "Testing procedures",
        "Problem-solving",
        "Communication",
        "Analytical skills",
        "Project management",
        "Teamwork",
        "Technical knowledge"
      ]
    },
    {
      "job_title": "Maintenance Supervisor",
      "skills": [
        "Equipment maintenance",
        "Preventive maintenance",
        "Troubleshooting",
        "Supervisory skills",
        "Safety protocols",
        "Budget management",
        "Communication",
        "Problem-solving",
        "Time management",
        "Teamwork",
        "Technical knowledge"
      ]
    },
    {
      "job_title": "Production Planner",
      "skills": [
        "Production scheduling",
        "Capacity planning",
        "Material planning",
        "Inventory management",
        "Data analysis",
        "SAP/ERP systems",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Organizational skills",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Industrial Mechanic",
      "skills": [
        "Equipment repair",
        "Maintenance",
        "Troubleshooting",
        "Blueprint reading",
        "Pneumatics",
        "Hydraulics",
        "Welding",
        "Electrical systems",
        "Safety protocols",
        "Problem-solving",
        "Communication",
        "Time management"
      ]
    },
    {
      "job_title": "Production Operator",
      "skills": [
        "Machine operation",
        "Quality control",
        "Standard operating procedures",
        "Safety protocols",
        "Attention to detail",
        "Troubleshooting",
        "Teamwork",
        "Communication",
        "Problem-solving",
        "Time management",
        "Physical stamina"
      ]
    },
    {
      "job_title": "Environmental Health and Safety Manager",
      "skills": [
        "Safety regulations",
        "Environmental regulations",
        "Risk assessment",
        "Incident investigation",
        "Safety training",
        "Emergency preparedness",
        "Regulatory compliance",
        "Auditing",
        "Communication",
        "Problem-solving",
        "Leadership",
        "Teamwork",
        "Analytical skills"
      ]
    },
    {
      "job_title": "Quality Technician",
      "skills": [
        "Quality inspections",
        "Testing procedures",
        "Data collection",
        "Documentation",
        "Root cause analysis",
        "Calibration",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Analytical skills",
        "Teamwork",
        "Technical knowledge"
      ]
    },
    {
      "job_title": "Production Engineer",
      "skills": [
        "Process optimization",
        "Production planning",
        "Data analysis",
        "Root cause analysis",
        "Continuous improvement",
        "Project management",
        "Problem-solving",
        "Communication",
        "Teamwork",
        "Leadership",
        "Technical knowledge",
        "Quality control"
      ]
    },
    {
      "job_title": "Shipping Supervisor",
      "skills": [
        "Shipping operations",
        "Inventory management",
        "Order processing",
        "Forklift operation",
        "Supervisory skills",
        "Safety protocols",
        "Communication",
        "Problem-solving",
        "Time management",
        "Attention to detail",
        "Teamwork",
        "ERP systems"
      ]
    },
    {
      "job_title": "Industrial Electrician",
      "skills": [
        "Electrical systems",
        "Equipment installation",
        "Troubleshooting",
        "PLC programming",
        "Blueprint reading",
        "Safety protocols",
        "Wiring",
        "Problem-solving",
        "Communication",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Materials Handler",
      "skills": [
        "Material handling",
        "Inventory control",
        "Forklift operation",
        "Order picking",
        "Shipping and receiving",
        "Safety protocols",
        "Attention to detail",
        "Communication",
        "Problem-solving",
        "Time management",
        "Organizational skills"
      ]
    },
    {
      "job_title": "Maintenance Electrician",
      "skills": [
        "Electrical systems",
        "Equipment maintenance",
        "Troubleshooting",
        "PLC programming",
        "Blueprint reading",
        "Safety protocols",
        "Wiring",
        "Problem-solving",
        "Communication",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Production Team Leader",
      "skills": [
        "Team leadership",
        "Production supervision",
        "Quality control",
        "Training and development",
        "Problem-solving",
        "Communication",
        "Time management",
        "Adaptability",
        "Decision-making",
        "Organizational skills",
        "Lean manufacturing"
      ]
    },
    {
      "job_title": "Materials Coordinator",
      "skills": [
        "Inventory management",
        "Material coordination",
        "Order processing",
        "SAP/ERP systems",
        "Data analysis",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Organizational skills",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Maintenance Mechanic",
      "skills": [
        "Equipment repair",
        "Preventive maintenance",
        "Troubleshooting",
        "Mechanical systems",
        "Electrical systems",
        "Blueprint reading",
        "Safety protocols",
        "Problem-solving",
        "Communication",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Continuous Improvement Specialist",
      "skills": [
        "Continuous improvement",
        "Lean manufacturing",
        "Six Sigma",
        "Process optimization",
        "Data analysis",
        "Problem-solving",
        "Project management",
        "Communication",
        "Leadership",
        "Teamwork",
        "Change management"
      ]
    },
    {
      "job_title": "Maintenance Planner",
      "skills": [
        "Maintenance planning",
        "Work order management",
        "Scheduling",
        "Inventory management",
        "SAP/ERP systems",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Organizational skills",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Process Operator",
      "skills": [
        "Process monitoring",
        "Equipment operation",
        "Safety protocols",
        "Quality control",
        "Troubleshooting",
        "Standard operating procedures",
        "Attention to detail",
        "Communication",
        "Problem-solving",
        "Teamwork",
        "Time management"
      ]
    },
    {
      "job_title": "Maintenance Manager",
      "skills": [
        "Maintenance management",
        "Equipment maintenance",
        "Preventive maintenance",
        "Troubleshooting",
        "Budget management",
        "Safety protocols",
        "Supervisory skills",
        "Communication",
        "Problem-solving",
        "Time management",
        "Teamwork",
        "Technical knowledge"
      ]
    },
    {
      "job_title": "Materials Analyst",
      "skills": [
        "Inventory management",
        "Demand forecasting",
        "Data analysis",
        "Supplier coordination",
        "SAP/ERP systems",
        "Problem-solving",
        "Communication",
        "Attention to detail",
        "Analytical skills",
        "Teamwork",
        "Time management"
      ]
    },
    {
      "job_title": "Quality Assurance Manager",
      "skills": [
        "Quality assurance",
        "Quality management systems",
        "ISO standards",
        "Auditing",
        "Root cause analysis",
        "Problem-solving",
        "Regulatory compliance",
        "Data analysis",
        "Communication",
        "Leadership",
        "Teamwork",
        "Process improvement"
      ]
    },
    {
      "job_title": "Maintenance Technician",
      "skills": [
        "Equipment maintenance",
        "Troubleshooting",
        "Repair",
        "Preventive maintenance",
        "Safety protocols",
        "Blueprint reading",
        "Electrical systems",
        "Mechanical systems",
        "Problem-solving",
        "Communication",
        "Time management",
        "Teamwork"
      ]
    },
    {
      "job_title": "Operations Manager",
      "skills": [
        "Operations management",
        "Strategic planning",
        "Budget management",
        "Process improvement",
        "Leadership",
        "Team management",
        "Problem-solving",
        "Communication",
        "Decision-making",
        "Analytical skills",
        "Project management",
        "Change management"
      ]
    }
  ]

    # Add more job titles and skills here

# Preprocess the data and create TaggedDocuments
tagged_data = [TaggedDocument(words=skills, tags = [job]) for skills in data]
#tagged_data = [TaggedDocument(words=skills, tags=[job]) for job, skills in data]

# Train the Doc2Vec model
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Test the model
skills = ["Project management", "Change management"]
infer_vector = model.infer_vector(skills)
similar_jobs = model.dv.most_similar([infer_vector], topn=3)

print(f"Given skills: {skills}")
print("Most suitable jobs:")
for job, similarity in similar_jobs:
    print(f"- {job} (Similarity: {similarity:.2f})")


<built-in method keys of dict object at 0x7fabfcd66d80>
Given skills: ['Project management', 'Change management']
Most suitable jobs:
- job_title (Similarity: 0.04)


In [ ]:
# Interpretation

'''

The output indicates the most suitable jobs based on the given skills of "communication" and "creativity". Each job is followed by a similarity score, which indicates how similar the job is to the given skills.

Designer (Similarity: 0.14): The job of a Designer has the highest similarity score of 0.14, indicating that it is the most suitable job based on the given skills. The positive similarity score suggests a higher relevance between the skills of communication and creativity and the job of a Designer.
Teacher (Similarity: -0.03): The job of a Teacher has a similarity score of -0.03, which is lower than the Designer. The negative similarity score suggests a lower relevance between the skills and the job.

Based on these similarity scores, the model suggests that the job of a Designer is the most suitable match for the given skills of communication and creativity. However, it's important to note that the scores are relative, and the actual interpretation of "suitability" may depend on the specific context and criteria used in the model.
'''

'\n\nThe output indicates the most suitable jobs based on the given skills of "communication" and "creativity". Each job is followed by a similarity score, which indicates how similar the job is to the given skills.\n\nDesigner (Similarity: 0.14): The job of a Designer has the highest similarity score of 0.14, indicating that it is the most suitable job based on the given skills. The positive similarity score suggests a higher relevance between the skills of communication and creativity and the job of a Designer.\nTeacher (Similarity: -0.03): The job of a Teacher has a similarity score of -0.03, which is lower than the Designer. The negative similarity score suggests a lower relevance between the skills and the job.\n\nBased on these similarity scores, the model suggests that the job of a Designer is the most suitable match for the given skills of communication and creativity. However, it\'s important to note that the scores are relative, and the actual interpretation of "suitability" 

In [ ]:
# model.wv.vocab

# 2 Explanation of code above

In [ ]:
## 1. Importing the necessary libraries:

# from gensim.models import Doc2Vec
# from gensim.models.doc2vec import TaggedDocument

"""
These statements import the Doc2Vec class from the gensim.models module 
and the TaggedDocument class from the gensim.models.doc2vec module. 
These classes are needed for training and using the Doc2Vec model. 
"""


'\nThese statements import the Doc2Vec class from the gensim.models module \nand the TaggedDocument class from the gensim.models.doc2vec module. \nThese classes are needed for training and using the Doc2Vec model. \n'

In [ ]:
## 2. Defining the data:

# data = [
#     ("Teacher", ["communication", "empathy", "solution-oriented"]),
#     ("Engineer", ["problem-solving", "analytical", "technical"]),
#     ("Designer", ["creativity", "visualization", "attention to detail"]),
#     # Add more job titles and skills here
# ]

"""
-> Max will provide a full data set
"""

In [ ]:
## 3. Preprocessing the data and creating TaggedDocuments:

# tagged_data = [TaggedDocument(words=skills, tags=[job]) for job, skills in data]


"""
-> This line creates a list of TaggedDocument objects from the job titles and skills 
in the data list. Each TaggedDocument contains the words attribute, 
which represents the skills, and the tags attribute, which represents the job title.

"""

'\n-> This line creates a list of TaggedDocument objects from the job titles and skills in the data list. Each TaggedDocument contains the words attribute, which represents the skills, and the tags attribute, which represents the job title.\n\n\n'

In [ ]:
# 4. Building and training the Doc2Vec model:


# model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
# model.build_vocab(tagged_data)
# model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

"""
Here, the Doc2Vec model is initialized with specific parameters:
 -vector_size: The dimensionality of the feature vectors.
 -window: The maximum distance between the predicted word and the context words.
 -min_count: The minimum count of words to consider during training.
 -workers: The number of worker threads to use during training.
 -epochs: The number of iterations (epochs) to train the model.
 -The model is then built and trained using the build_vocab() and train() methods, respectively. The build_vocab() method initializes the vocabulary from the provided tagged_data, and the train() method trains the model on the tagged_data.
"""

In [ ]:
# 5. Testing the model:

# skills = ["communication", "creativity"]
# infer_vector = model.infer_vector(skills)
# similar_jobs = model.dv.most_similar([infer_vector], topn=3)

"""
In this part, a list of skills is defined (skills). 
The infer_vector() method is used to infer a vector representation for the given skills using the trained model. 
This vector representation captures the semantic meaning of the skills.

The most_similar() method of the model.dv (document vectors) is then called to find the most similar job titles 
to the inferred vector. The topn parameter specifies the number of similar jobs to retrieve.
"""

In [ ]:
# 6. Printing the results:

# print(f"Given skills: {skills}")
# print("Most suitable jobs:")
# for job, similarity in similar_jobs:
#     print(f"- {job} (Similarity: {similarity:.2f})")

"""
Finally, the given skills are printed, followed by the most suitable job titles 
and their similarity scores. The similarity scores indicate how similar each job title is to the given skills, 
with higher
"""


'\nFinally, the given skills are printed, followed by the most suitable job titles \nand their similarity scores. The similarity scores indicate how similar each job title is to the given skills, \nwith higher\n'

# 3 Weitere Tests zum Ausprobieren

In [ ]:
# from gensim.models import Word2Vec

# # Assuming you have a list of job descriptions or skill-related information
# job_descriptions = [
#     "Software Engineer with expertise in Python and machine learning",
#     "Data Analyst proficient in SQL and data visualization",
#     "Web Developer skilled in HTML, CSS, and JavaScript",
#     # Add more job descriptions...
# ]

# # Preprocess the job descriptions (tokenization, lowercasing, etc.)
# preprocessed_job_descriptions = [job.lower().split() for job in job_descriptions]

# # Train the Skill2Vec model
# skill2vec_model = Word2Vec(preprocessed_job_descriptions, sg=1, window=5, min_count=1)

# # Save the trained model
# skill2vec_model.save("path_to_save_skill2vec_model")


In [ ]:
# In this code, we assume that you have already trained a Skill2Vec model using the gensim library
# You would replace "path_to_skill2vec_model" with the actual path to your pre-trained model file

# The find_best_job_title function takes a list of skills as input and iterates over all the available job titles in the Skill2Vec model
# For each job title, it computes the similarity between the job title's vector and the vectors of the input skills

# The similarity is calculated as the dot product between the skill vectors and the job title vector.
# The function keeps track of the job title with the highest similarity and returns it as the best match.



In [ ]:
# from gensim.models import Word2Vec

# # Assuming you have a pre-trained Skill2Vec model
# skill2vec_model = Word2Vec.load("path_to_skill2vec_model")

# def find_best_job_title(skills):
#     job_titles = skill2vec_model.wv.vocab.keys()
#     skill_vectors = [skill2vec_model.wv[skill] for skill in skills]

#     best_job_title = None
#     best_similarity = -1

#     for job_title in job_titles:
#         job_vector = skill2vec_model.wv[job_title]
#         similarity = sum(skill_vector.dot(job_vector) for skill_vector in skill_vectors)
        
#         if similarity > best_similarity:
#             best_similarity = similarity
#             best_job_title = job_title
    
#     return best_job_title

# # Example usage
# skills = ["python", "machine_learning", "data_analysis"]
# best_job = find_best_job_title(skills)
# print("Best job title:", best_job)
